# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [3]:
# Install the requirements in Google Colab
%pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\7sart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)



# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-NomadevTry"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [7]:
# Let's test the base model before training
prompt = "Whats the color for this hexcode : #0066cc"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color for this hexcode : #0066cc
Whats the color


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [11]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="burkelibbey/colors")

print(ds["train"][0])

{'color': '#000000', 'description': 'Pure Black: A shade that completely absorbs light and does not reflect any colors. It is the darkest possible shade.'}


In [22]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

# Preprocess the dataset
def process_dataset(sample):
    chatml_format = (
        f"<|im_start|>user\n{sample['color']}\n<|im_end|>\n"
        f"<|im_start|>assistant\n{sample['description']}\n<|im_end|>\n"
    )
    return {"chatml_format": chatml_format}

ds = ds.map(process_dataset)

# Tokenize the dataset
def tokenize_function(sample):
    return tokenizer(
        sample["chatml_format"],
        truncation=True,
        padding=True,
        max_length=512,
    )

ds = ds.map(tokenize_function, batched=True)

# Split the dataset
split_ds = ds["train"].train_test_split(test_size=0.2)
train_dataset = split_ds["train"]
test_dataset = split_ds["test"]



Map:   0%|          | 0/33887 [00:00<?, ? examples/s]

Map:   0%|          | 0/33887 [00:00<?, ? examples/s]

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [25]:
# Configure SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=100,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    max_seq_length=512,  # Explicitly set the sequence length
    use_mps_device=(True if device == "mps" else False),
    hub_model_id="Smol2_SFT_color",
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=test_dataset,
    dataset_text_field="chatml_format",  # Specify the correct dataset field
)

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [37]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [31]:
trainer.push_to_hub(tags=finetune_tags)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/thenomdevel/Smol2_SFT_color/commit/424b8ce2bd3c968b44c098c81d4458f69616f486', commit_message='End of training', commit_description='', oid='424b8ce2bd3c968b44c098c81d4458f69616f486', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thenomdevel/Smol2_SFT_color', endpoint='https://huggingface.co', repo_type='model', repo_id='thenomdevel/Smol2_SFT_color'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [43]:
# Test the fine-tuned model on the same prompt
model_name = "thenomdevel/Smol2_SFT_color"  # Your uploaded model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# Let's test the base model before training
prompt = "#ffee60"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=10)  # Limit response length
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated response
print("Response:")
print(response)

Response:
user
#ffee60
#Colorado
#Blue
#Light Blue


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.